Setting up the connection with the Google Collab notebook

In [2]:
# connect information in the google drive to this collab session and change the working directory
from google.colab import drive
drive.mount('/content/drive/')
# Changing the working directory to the Assignment folder
import os
os.chdir('/content/drive/MyDrive/CA684_Assignment/')
!ls # Listing the files in working directory

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Additional_Resources  Complete_Sources	Model_Scores.gsheet  Tutorials
Assignment_Overview   Dev-set		Test-set


Importing and loading the required set of packages

In [3]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
!pip install pyprind
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import pyprind
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import xgboost as xgb
from keras import Sequential
from keras import preprocessing
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras import regularizers

     \ 225kB 1.5MB/s
  Created wheel for tqdm: filename=tqdm-4.28.1-py2.py3-none-any.whl size=47868 sha256=89f5c44747efbecbd44bfe73621b7da1dfd86553e13a6c495058720d146f72af
  Stored in directory: /tmp/pip-ephem-wheel-cache-258fnlm1/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.28.1 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement tqdm>=4.36.1, but you'll have tqdm 4.28.1 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


Functions used for loading the feature set and calculating spearman score

In [4]:
# Function to calculate the Spearman Correlation score
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

# Function to read C3D feature
def read_C3D(fname):
    """Scan vectors from file"""
    with open(fname) as f:
        for line in f:
            C3D =[float(item) for item in line.split()] # convert to float type, using default separator
    return C3D
  
# Function to read HMP feature
def read_HMP(fname):
    """Scan HMP(Histogram of Motion Patterns) features from file"""
    with open(fname) as f:
        for line in f:
            pairs=line.split()
            HMP_temp = { int(p.split(':')[0]) : float(p.split(':')[1]) for p in pairs}
    # there are 6075 bins, fill zeros
    HMP = np.zeros(6075)
    for idx in HMP_temp.keys():
        HMP[idx-1] = HMP_temp[idx]            
    return HMP

# Function to read LBP feature
def read_LBP(fname):
    """Scan vectors from file"""
    with open(fname) as f:
        for line in f:
            LBP =[float(item) for item in line.split()] # convert to float type, using default separator
    return LBP


Loading the ground truth file from the Dev set

In [5]:
# load the ground truth values
GT = pd.read_csv('./Dev-set/Ground-truth/ground-truth.csv')
vid = GT.video.values  # list the video filenames from the ground truth 

In [6]:
GT.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


Loading the features into a python dataframe from the drive (one time load)

In [ ]:
# Extracting the C3D feature
c3d_features = pd.DataFrame(columns=['video','C3D'])
for i in range(6000):
  c3d_features = c3d_features.append({'video': vid[i], 'C3D': read_C3D('./Dev-set/C3D/' + vid[i].split('.')[0] + '.txt')}, ignore_index=True)
c3d_features.shape

(6000, 2)

In [ ]:
# Extracting the HMP feature
hmp_features = pd.DataFrame(columns=['video','HMP'])
for i in range(6000):
  hmp_features = hmp_features.append({'video': vid[i], 'HMP': read_HMP('./Dev-set/HMP/' + vid[i].split('.')[0] + '.txt')}, ignore_index=True)
hmp_features.shape

(6000, 2)

In [44]:
# Extracting the LBP feature
lbp_features1 = pd.DataFrame(columns=['video','LBP'])
lbp_features2 = pd.DataFrame(columns=['video','LBP'])
lbp_features3 = pd.DataFrame(columns=['video','LBP'])
for i in range(6000):
  lbp_features1 = lbp_features1.append({'video': vid[i], 'LBP': read_LBP('./Dev-set/LBP/' + vid[i].split('.')[0] + '-0.txt')}, ignore_index=True)
  lbp_features2 = lbp_features2.append({'video': vid[i], 'LBP': read_LBP('./Dev-set/LBP/' + vid[i].split('.')[0] + '-56.txt')}, ignore_index=True)
  lbp_features3 = lbp_features3.append({'video': vid[i], 'LBP': read_LBP('./Dev-set/LBP/' + vid[i].split('.')[0] + '-112.txt')}, ignore_index=True)
lbp_features1.shape
lbp_features2.shape
lbp_features3.shape

(6000, 2)

Saving and Loading the extracted features using the pickle files

In [ ]:
#----------- Saving the features to the drive ------------

os.chdir('/content/drive/MyDrive')
# Saving the C3D feature in Google Drive to reuse later  
c3d_features.to_pickle('./C3D_Features.pickle')
# Saving the HMP feature in Google Drive to reuse later  
hmp_features.to_pickle('./HMP_Features.pickle')
# Saving the LBP 0 frame feature in Google Drive to reuse later  
lbp_features1.to_pickle('./LBP_0Frame_Features.pickle')
# Saving the LBP 56 frame feature in Google Drive to reuse later  
lbp_features2.to_pickle('./LBP_56Frame_Features.pickle')
# Saving the LBP 112 frame feature in Google Drive to reuse later  
lbp_features3.to_pickle('./LBP_112Frame_Features.pickle')
# Changing the working directory
os.chdir('/content/drive/MyDrive/CA684_Assignment/')

In [7]:
#----------- Loading the features from the drive ------------

os.chdir('/content/drive/MyDrive')
# Load C3D feature back in memory session
c3d_features = pd.read_pickle('./C3D_Features.pickle')
# Load HMP feature back in memory session
hmp_features = pd.read_pickle('./HMP_Features.pickle')
# Load LBP frame features back in memory session
lbp_0frame_features = pd.read_pickle('./LBP_0Frame_Features.pickle')
lbp_56frame_features = pd.read_pickle('./LBP_56Frame_Features.pickle')
lbp_112frame_features = pd.read_pickle('./LBP_112Frame_Features.pickle')
# Changing the working directory
os.chdir('/content/drive/MyDrive/CA684_Assignment/')

Extracting the values of features from 1-D numpy array

In [8]:
# C3D feature value extracted
sequences = c3d_features['C3D']
C3D_seq = np.zeros((len(sequences),len(c3d_features.loc[0,'C3D'])))
for i in range(len(sequences)):
    n = len(sequences[i])
    C3D_seq[i,-n:] = sequences[i]
C3D_seq.shape

(6000, 101)

In [9]:
# HMP feature value extracted
sequences = hmp_features['HMP']
HMP_seq = np.zeros((len(sequences),len(hmp_features.loc[0,'HMP'])))
for i in range(len(sequences)):
    n = len(sequences[i])
    HMP_seq[i,-n:] = sequences[i]
HMP_seq.shape

(6000, 6075)

In [10]:
# LBP feature value extracted
def LBP_Frame(lbp_features) :
  sequences = lbp_features['LBP']
  LBP_seq = np.zeros((len(sequences),len(lbp_features.loc[0,'LBP'])))
  for i in range(len(sequences)):
      n = len(sequences[i])
      LBP_seq[i,-n:] = sequences[i]
  return LBP_seq
LBP_seq = np.concatenate((LBP_Frame(lbp_0frame_features),LBP_Frame(lbp_56frame_features),LBP_Frame(lbp_112frame_features)),axis = 1)
LBP_seq.shape

(6000, 366)

PCA technique to reduce the dimensionality of HMP feature

In [28]:
from sklearn.decomposition import PCA
#pca = PCA(n_components=256)
# preserving the 99.9% of the variance i.e. reduced components explains 99.9% variance of the data 
pca = PCA(0.999)
HMP_seq_PCA = pd.DataFrame(pca.fit_transform(HMP_seq))
HMP_seq_PCA.shape

(6000, 178)

Merging the feature set 

In [29]:
# merging the C3D and reduced HMP feature set
C3D_HMP = np.concatenate((C3D_seq,HMP_seq_PCA), axis = 1)

# merging the C3D and LBP feature set
C3D_LBP = np.concatenate((C3D_seq,LBP_seq), axis = 1)

# merging the C3D, LBP and reduced HMP feature set
C3D_LBP_HMP = np.concatenate((C3D_seq,LBP_seq,HMP_seq_PCA), axis = 1)

print(C3D_HMP.shape)
print(C3D_LBP.shape)
print(C3D_LBP_HMP.shape)

(6000, 279)
(6000, 467)
(6000, 645)


Feature set used to predict the memorability and spearman score
1. C3D
2. HMP
3. LBP
4. C3D & HMP
5. C3D & LBP
6. C3D, LBP & HMP

Models used to check the spearman score
1. K nearest neighbour
2. Random forest
3. Support vector regressor
4. Nueral network
5. Gradient boosting regressor

###Using C3D feature

Setting up the DV (Y) and IDVs (X) along with train test split

In [132]:
Y = GT[['short-term_memorability','long-term_memorability']].values
st_Y = GT[['short-term_memorability']].values
lt_Y = GT[['long-term_memorability']].values
X = C3D_seq 
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state = 42) 
st_X_train,st_X_test,st_Y_train,st_Y_test = train_test_split(X,st_Y,test_size=0.2,random_state=40)
lt_X_train,lt_X_test,lt_Y_train,lt_Y_test = train_test_split(X,lt_Y,test_size=0.2,random_state=40)

KNN model

In [51]:
knnRegressor = KNeighborsRegressor(n_neighbors = 77)
knnRegressor.fit(X_train, Y_train)
y_pred = knnRegressor.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.251
The Spearman's correlation coefficient is: 0.097


Random Forest ensemble model

In [52]:
rf = RandomForestRegressor(n_estimators=100,random_state=42,verbose=2)
rf.fit(X_train,Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   43.2s finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=2, warm_start=False)

In [53]:
rf_pred = rf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [55]:
Get_score(rf_pred, Y_test)

The Spearman's correlation coefficient is: 0.314
The Spearman's correlation coefficient is: 0.117


Building Neural Models from Keras

In [58]:
# add dropout - Helps prevent overfitting
# add regularizers - Regularizers allow to apply penalties on layer parameters or layer activity during optimization.
# activations - Used to determine the output of network. It maps the resulting values in between 0 to 1 or -1 to 1 etc. (depending upon the function).
model = Sequential() # The Sequential model is a linear stack of layers.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005),input_shape=(101,))) # Just your regular densely-connected NN layer.
model.add(layers.Dropout(0.5)) #Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training; helps prevent overfitting.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

In [59]:
# compile the model 
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])
# training the model 
history = model.fit(X_train,Y_train,epochs=20)
predictions = model.predict(X_test)
Get_score(predictions, Y_test)

Epoch 1/20
150/150 [==============================] - 1s 1ms/step - loss: 0.0984 - accuracy: 0.6621
Epoch 2/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0330 - accuracy: 0.6820
Epoch 3/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0237 - accuracy: 0.6988
Epoch 4/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0201 - accuracy: 0.7004
Epoch 5/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0187 - accuracy: 0.6944
Epoch 6/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0176 - accuracy: 0.6962
Epoch 7/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0159 - accuracy: 0.6950
Epoch 8/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0153 - accuracy: 0.7031
Epoch 9/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0148 - accuracy: 0.7022
Epoch 10/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0145 - accuracy: 0.6988

SVR model

In [60]:
st_X = StandardScaler()
st_Y = StandardScaler()
st_X_train = st_X.fit_transform(st_X_train)
st_Y_train = st_Y.fit_transform(st_Y_train)
lt_X = StandardScaler()
lt_Y = StandardScaler()
lt_X_train = lt_X.fit_transform(lt_X_train)
lt_Y_train = lt_Y.fit_transform(lt_Y_train)

In [61]:
st_regressor = SVR(kernel = 'rbf')
st_regressor.fit(st_X_train, st_Y_train)
lt_regressor = SVR(kernel = 'rbf')
lt_regressor.fit(lt_X_train,lt_Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [62]:
st_pred = st_regressor.predict(st_X_test)
st_pred = st_Y.inverse_transform(st_pred)
lt_pred = lt_regressor.predict(lt_X_test)
lt_pred = lt_Y.inverse_transform(lt_pred)

In [63]:
Get_score(st_pred, st_Y_test)
Get_score(lt_pred, lt_Y_test)

The Spearman's correlation coefficient is: 0.242
The Spearman's correlation coefficient is: 0.107


XGBoost model

In [133]:
params = {'n_estimators':650, 'max_depth':12, 'min_samples_split':2, 'learning_rate':0.01, 'loss':'lad'}
xgb_st = xgb.XGBRegressor(**params)
xgb_lt = xgb.XGBRegressor(**params)
xgb_st.fit(st_X_train, st_Y_train)
xgb_lt.fit(lt_X_train, lt_Y_train)

[13:44:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:45:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, loss='lad',
             max_delta_step=0, max_depth=12, min_child_weight=1,
             min_samples_split=2, missing=None, n_estimators=650, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [134]:
Get_score(xgb_st.predict(st_X_test), st_Y_test)
Get_score(xgb_lt.predict(lt_X_test), lt_Y_test)

The Spearman's correlation coefficient is: 0.295
The Spearman's correlation coefficient is: 0.071


## Using HMP feature

Setting up the DV (Y) and IDVs (X) along with train test split

In [135]:
Y = GT[['short-term_memorability','long-term_memorability']].values
st_Y = GT[['short-term_memorability']].values
lt_Y = GT[['long-term_memorability']].values
X = HMP_seq_PCA 
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state = 42) 
st_X_train,st_X_test,st_Y_train,st_Y_test = train_test_split(X,st_Y,test_size=0.2,random_state=40)
lt_X_train,lt_X_test,lt_Y_train,lt_Y_test = train_test_split(X,lt_Y,test_size=0.2,random_state=40)

KNN model

In [31]:
knnRegressor = KNeighborsRegressor(n_neighbors = 77)
knnRegressor.fit(X_train, Y_train)
y_pred = knnRegressor.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.237
The Spearman's correlation coefficient is: 0.075


Random Forest ensemble model

In [34]:
rf = RandomForestRegressor(n_estimators=100,random_state=42,verbose=2)
rf.fit(X_train,Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.4min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=2, warm_start=False)

In [36]:
rf_pred = rf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [38]:
Get_score(rf_pred, Y_test)

The Spearman's correlation coefficient is: 0.247
The Spearman's correlation coefficient is: 0.065


Building Neural Models from Keras

In [39]:
# add dropout - Helps prevent overfitting
# add regularizers - Regularizers allow to apply penalties on layer parameters or layer activity during optimization.
# activations - Used to determine the output of network. It maps the resulting values in between 0 to 1 or -1 to 1 etc. (depending upon the function).
model = Sequential() # The Sequential model is a linear stack of layers.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005),input_shape=(178,))) # Just your regular densely-connected NN layer.
model.add(layers.Dropout(0.5)) #Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training; helps prevent overfitting.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

In [40]:
# compile the model 
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])
# training the model 
history = model.fit(X_train,Y_train,epochs=20)
predictions = model.predict(X_test)
Get_score(predictions, Y_test)

Epoch 1/20
150/150 [==============================] - 1s 1ms/step - loss: 0.1063 - accuracy: 0.5255
Epoch 2/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0459 - accuracy: 0.5138
Epoch 3/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0303 - accuracy: 0.5499
Epoch 4/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0217 - accuracy: 0.6020
Epoch 5/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0187 - accuracy: 0.6233
Epoch 6/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0169 - accuracy: 0.6470
Epoch 7/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0163 - accuracy: 0.6934
Epoch 8/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0153 - accuracy: 0.6985
Epoch 9/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0151 - accuracy: 0.7201
Epoch 10/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0146 - accuracy: 0.6984

SVR model

In [41]:
st_X = StandardScaler()
st_Y = StandardScaler()
st_X_train = st_X.fit_transform(st_X_train)
st_Y_train = st_Y.fit_transform(st_Y_train)
lt_X = StandardScaler()
lt_Y = StandardScaler()
lt_X_train = lt_X.fit_transform(lt_X_train)
lt_Y_train = lt_Y.fit_transform(lt_Y_train)

In [42]:
st_regressor = SVR(kernel = 'rbf')
st_regressor.fit(st_X_train, st_Y_train)
lt_regressor = SVR(kernel = 'rbf')
lt_regressor.fit(lt_X_train,lt_Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [43]:
st_pred = st_regressor.predict(st_X_test)
st_pred = st_Y.inverse_transform(st_pred)
lt_pred = lt_regressor.predict(lt_X_test)
lt_pred = lt_Y.inverse_transform(lt_pred)

In [44]:
Get_score(st_pred, st_Y_test)
Get_score(lt_pred, lt_Y_test)

The Spearman's correlation coefficient is: 0.199
The Spearman's correlation coefficient is: 0.087


XGBoost Model 

In [136]:
import xgboost as xgb
params = {'n_estimators':650, 'max_depth':12, 'min_samples_split':2, 'learning_rate':0.01, 'loss':'lad'}
xgb_st = xgb.XGBRegressor(**params)
xgb_lt = xgb.XGBRegressor(**params)
xgb_st.fit(st_X_train, st_Y_train)
xgb_lt.fit(lt_X_train, lt_Y_train)

[13:48:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:50:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, loss='lad',
             max_delta_step=0, max_depth=12, min_child_weight=1,
             min_samples_split=2, missing=None, n_estimators=650, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [137]:
Get_score(xgb_st.predict(st_X_test), st_Y_test)
Get_score(xgb_lt.predict(lt_X_test), lt_Y_test)

The Spearman's correlation coefficient is: 0.268
The Spearman's correlation coefficient is: 0.073


## Using LBP feature

Setting up the DV (Y) and IDVs (X) along with train test split

In [139]:
Y = GT[['short-term_memorability','long-term_memorability']].values
st_Y = GT[['short-term_memorability']].values
lt_Y = GT[['long-term_memorability']].values
X = LBP_seq 
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state = 42) 
st_X_train,st_X_test,st_Y_train,st_Y_test = train_test_split(X,st_Y,test_size=0.2,random_state=40)
lt_X_train,lt_X_test,lt_Y_train,lt_Y_test = train_test_split(X,lt_Y,test_size=0.2,random_state=40)

KNN model

In [72]:
knnRegressor = KNeighborsRegressor(n_neighbors = 77)
knnRegressor.fit(X_train, Y_train)
y_pred = knnRegressor.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.230
The Spearman's correlation coefficient is: 0.055


Random Forest ensemble model

In [73]:
rf = RandomForestRegressor(n_estimators=100,random_state=42,verbose=2)
rf.fit(X_train,Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.9min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=2, warm_start=False)

In [74]:
rf_pred = rf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [75]:
Get_score(rf_pred, Y_test)

The Spearman's correlation coefficient is: 0.318
The Spearman's correlation coefficient is: 0.057


Building Neural Models from Keras

In [76]:
# add dropout - Helps prevent overfitting
# add regularizers - Regularizers allow to apply penalties on layer parameters or layer activity during optimization.
# activations - Used to determine the output of network. It maps the resulting values in between 0 to 1 or -1 to 1 etc. (depending upon the function).
model = Sequential() # The Sequential model is a linear stack of layers.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005),input_shape=(366,))) # Just your regular densely-connected NN layer.
model.add(layers.Dropout(0.5)) #Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training; helps prevent overfitting.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

In [77]:
# compile the model 
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])
# training the model 
history = model.fit(X_train,Y_train,epochs=20)
predictions = model.predict(X_test)
Get_score(predictions, Y_test)

Epoch 1/20
150/150 [==============================] - 1s 1ms/step - loss: 0.0990 - accuracy: 0.5958
Epoch 2/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0397 - accuracy: 0.6162
Epoch 3/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0254 - accuracy: 0.6611
Epoch 4/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0198 - accuracy: 0.6766
Epoch 5/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0173 - accuracy: 0.6981
Epoch 6/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0158 - accuracy: 0.7053
Epoch 7/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0159 - accuracy: 0.7020
Epoch 8/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0156 - accuracy: 0.7062
Epoch 9/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0146 - accuracy: 0.7009
Epoch 10/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0148 - accuracy: 0.7123

SVR model

In [78]:
st_X = StandardScaler()
st_Y = StandardScaler()
st_X_train = st_X.fit_transform(st_X_train)
st_Y_train = st_Y.fit_transform(st_Y_train)
lt_X = StandardScaler()
lt_Y = StandardScaler()
lt_X_train = lt_X.fit_transform(lt_X_train)
lt_Y_train = lt_Y.fit_transform(lt_Y_train)

In [79]:
st_regressor = SVR(kernel = 'rbf')
st_regressor.fit(st_X_train, st_Y_train)
lt_regressor = SVR(kernel = 'rbf')
lt_regressor.fit(lt_X_train,lt_Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [80]:
st_pred = st_regressor.predict(st_X_test)
st_pred = st_Y.inverse_transform(st_pred)
lt_pred = lt_regressor.predict(lt_X_test)
lt_pred = lt_Y.inverse_transform(lt_pred)

In [81]:
Get_score(st_pred, st_Y_test)
Get_score(lt_pred, lt_Y_test)

The Spearman's correlation coefficient is: -0.088
The Spearman's correlation coefficient is: -0.040


XGBoost model

In [140]:
params = {'n_estimators':650, 'max_depth':12, 'min_samples_split':2, 'learning_rate':0.01, 'loss':'lad'}
xgb_st = xgb.XGBRegressor(**params)
xgb_lt = xgb.XGBRegressor(**params)
xgb_st.fit(st_X_train, st_Y_train)
xgb_lt.fit(lt_X_train, lt_Y_train)

[13:58:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:02:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, loss='lad',
             max_delta_step=0, max_depth=12, min_child_weight=1,
             min_samples_split=2, missing=None, n_estimators=650, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [141]:
Get_score(xgb_st.predict(st_X_test), st_Y_test)
Get_score(xgb_lt.predict(lt_X_test), lt_Y_test)

The Spearman's correlation coefficient is: 0.253
The Spearman's correlation coefficient is: 0.051


## Using C3D & HMP feature

Setting up the DV (Y) and IDVs (X) along with train test split

In [142]:
Y = GT[['short-term_memorability','long-term_memorability']].values
st_Y = GT[['short-term_memorability']].values
lt_Y = GT[['long-term_memorability']].values
X = C3D_HMP 
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state = 42) 
st_X_train,st_X_test,st_Y_train,st_Y_test = train_test_split(X,st_Y,test_size=0.2,random_state=40)
lt_X_train,lt_X_test,lt_Y_train,lt_Y_test = train_test_split(X,lt_Y,test_size=0.2,random_state=40)

KNN model

In [87]:
knnRegressor = KNeighborsRegressor(n_neighbors = 77)
knnRegressor.fit(X_train, Y_train)
y_pred = knnRegressor.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.255
The Spearman's correlation coefficient is: 0.100


Random Forest ensemble model

In [88]:
rf = RandomForestRegressor(n_estimators=100,random_state=42,verbose=2)
rf.fit(X_train,Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.1min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=2, warm_start=False)

In [89]:
rf_pred = rf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [90]:
Get_score(rf_pred, Y_test)

The Spearman's correlation coefficient is: 0.303
The Spearman's correlation coefficient is: 0.123


Building Neural Models from Keras

In [93]:
# add dropout - Helps prevent overfitting
# add regularizers - Regularizers allow to apply penalties on layer parameters or layer activity during optimization.
# activations - Used to determine the output of network. It maps the resulting values in between 0 to 1 or -1 to 1 etc. (depending upon the function).
model = Sequential() # The Sequential model is a linear stack of layers.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005),input_shape=(279,))) # Just your regular densely-connected NN layer.
model.add(layers.Dropout(0.5)) #Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training; helps prevent overfitting.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

In [94]:
# compile the model 
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])
# training the model 
history = model.fit(X_train,Y_train,epochs=20)
predictions = model.predict(X_test)
Get_score(predictions, Y_test)

Epoch 1/20
150/150 [==============================] - 1s 1ms/step - loss: 0.1021 - accuracy: 0.6649
Epoch 2/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0370 - accuracy: 0.6504
Epoch 3/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0247 - accuracy: 0.6678
Epoch 4/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0195 - accuracy: 0.6928
Epoch 5/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0179 - accuracy: 0.6868
Epoch 6/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0169 - accuracy: 0.7012
Epoch 7/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0154 - accuracy: 0.6932
Epoch 8/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0152 - accuracy: 0.7047
Epoch 9/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0149 - accuracy: 0.6998
Epoch 10/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0146 - accuracy: 0.6938

SVR model

In [95]:
st_X = StandardScaler()
st_Y = StandardScaler()
st_X_train = st_X.fit_transform(st_X_train)
st_Y_train = st_Y.fit_transform(st_Y_train)
lt_X = StandardScaler()
lt_Y = StandardScaler()
lt_X_train = lt_X.fit_transform(lt_X_train)
lt_Y_train = lt_Y.fit_transform(lt_Y_train)

In [96]:
st_regressor = SVR(kernel = 'rbf')
st_regressor.fit(st_X_train, st_Y_train)
lt_regressor = SVR(kernel = 'rbf')
lt_regressor.fit(lt_X_train,lt_Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [97]:
st_pred = st_regressor.predict(st_X_test)
st_pred = st_Y.inverse_transform(st_pred)
lt_pred = lt_regressor.predict(lt_X_test)
lt_pred = lt_Y.inverse_transform(lt_pred)

In [98]:
Get_score(st_pred, st_Y_test)
Get_score(lt_pred, lt_Y_test)

The Spearman's correlation coefficient is: 0.259
The Spearman's correlation coefficient is: 0.137


XGBoost model

In [143]:
params = {'n_estimators':650, 'max_depth':12, 'min_samples_split':2, 'learning_rate':0.01, 'loss':'lad'}
xgb_st = xgb.XGBRegressor(**params)
xgb_lt = xgb.XGBRegressor(**params)
xgb_st.fit(st_X_train, st_Y_train)
xgb_lt.fit(lt_X_train, lt_Y_train)

[14:08:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:11:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, loss='lad',
             max_delta_step=0, max_depth=12, min_child_weight=1,
             min_samples_split=2, missing=None, n_estimators=650, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [144]:
Get_score(xgb_st.predict(st_X_test), st_Y_test)
Get_score(xgb_lt.predict(lt_X_test), lt_Y_test)

The Spearman's correlation coefficient is: 0.313
The Spearman's correlation coefficient is: 0.077


## Using C3D & LBP feature

Setting up the DV (Y) and IDVs (X) along with train test split

In [145]:
Y = GT[['short-term_memorability','long-term_memorability']].values
st_Y = GT[['short-term_memorability']].values
lt_Y = GT[['long-term_memorability']].values
X = C3D_LBP 
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state = 42) 
st_X_train,st_X_test,st_Y_train,st_Y_test = train_test_split(X,st_Y,test_size=0.2,random_state=40)
lt_X_train,lt_X_test,lt_Y_train,lt_Y_test = train_test_split(X,lt_Y,test_size=0.2,random_state=40)

KNN model

In [102]:
knnRegressor = KNeighborsRegressor(n_neighbors = 77)
knnRegressor.fit(X_train, Y_train)
y_pred = knnRegressor.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.274
The Spearman's correlation coefficient is: 0.104


Random Forest ensemble model

In [103]:
rf = RandomForestRegressor(n_estimators=100,random_state=42,verbose=2)
rf.fit(X_train,Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  3.7min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=2, warm_start=False)

In [104]:
rf_pred = rf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [105]:
Get_score(rf_pred, Y_test)

The Spearman's correlation coefficient is: 0.333
The Spearman's correlation coefficient is: 0.099


Building Neural Models from Keras

In [106]:
# add dropout - Helps prevent overfitting
# add regularizers - Regularizers allow to apply penalties on layer parameters or layer activity during optimization.
# activations - Used to determine the output of network. It maps the resulting values in between 0 to 1 or -1 to 1 etc. (depending upon the function).
model = Sequential() # The Sequential model is a linear stack of layers.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005),input_shape=(467,))) # Just your regular densely-connected NN layer.
model.add(layers.Dropout(0.5)) #Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training; helps prevent overfitting.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

In [107]:
# compile the model 
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])
# training the model 
history = model.fit(X_train,Y_train,epochs=20)
predictions = model.predict(X_test)
Get_score(predictions, Y_test)

Epoch 1/20
150/150 [==============================] - 1s 1ms/step - loss: 0.0938 - accuracy: 0.5997
Epoch 2/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0392 - accuracy: 0.6171
Epoch 3/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0269 - accuracy: 0.6396
Epoch 4/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0221 - accuracy: 0.6547
Epoch 5/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0187 - accuracy: 0.6816
Epoch 6/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0175 - accuracy: 0.6798
Epoch 7/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0161 - accuracy: 0.7024
Epoch 8/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0154 - accuracy: 0.7115
Epoch 9/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0149 - accuracy: 0.7120
Epoch 10/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0143 - accuracy: 0.7105

SVR model

In [108]:
st_X = StandardScaler()
st_Y = StandardScaler()
st_X_train = st_X.fit_transform(st_X_train)
st_Y_train = st_Y.fit_transform(st_Y_train)
lt_X = StandardScaler()
lt_Y = StandardScaler()
lt_X_train = lt_X.fit_transform(lt_X_train)
lt_Y_train = lt_Y.fit_transform(lt_Y_train)

In [109]:
st_regressor = SVR(kernel = 'rbf')
st_regressor.fit(st_X_train, st_Y_train)
lt_regressor = SVR(kernel = 'rbf')
lt_regressor.fit(lt_X_train,lt_Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [110]:
st_pred = st_regressor.predict(st_X_test)
st_pred = st_Y.inverse_transform(st_pred)
lt_pred = lt_regressor.predict(lt_X_test)
lt_pred = lt_Y.inverse_transform(lt_pred)

In [111]:
Get_score(st_pred, st_Y_test)
Get_score(lt_pred, lt_Y_test)

The Spearman's correlation coefficient is: 0.228
The Spearman's correlation coefficient is: 0.094


XGBoost model

In [146]:
params = {'n_estimators':650, 'max_depth':12, 'min_samples_split':2, 'learning_rate':0.01, 'loss':'lad'}
xgb_st = xgb.XGBRegressor(**params)
xgb_lt = xgb.XGBRegressor(**params)
xgb_st.fit(st_X_train, st_Y_train)
xgb_lt.fit(lt_X_train, lt_Y_train)

[14:22:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:27:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, loss='lad',
             max_delta_step=0, max_depth=12, min_child_weight=1,
             min_samples_split=2, missing=None, n_estimators=650, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [147]:
Get_score(xgb_st.predict(st_X_test), st_Y_test)
Get_score(xgb_lt.predict(lt_X_test), lt_Y_test)

The Spearman's correlation coefficient is: 0.324
The Spearman's correlation coefficient is: 0.095


## Using C3D, HMP & LBP feature

Setting up the DV (Y) and IDVs (X) along with train test split

In [148]:
Y = GT[['short-term_memorability','long-term_memorability']].values
st_Y = GT[['short-term_memorability']].values
lt_Y = GT[['long-term_memorability']].values
X = C3D_LBP_HMP
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state = 42) 
st_X_train,st_X_test,st_Y_train,st_Y_test = train_test_split(X,st_Y,test_size=0.2,random_state=40)
lt_X_train,lt_X_test,lt_Y_train,lt_Y_test = train_test_split(X,lt_Y,test_size=0.2,random_state=40)

KNN model

In [115]:
knnRegressor = KNeighborsRegressor(n_neighbors = 77)
knnRegressor.fit(X_train, Y_train)
y_pred = knnRegressor.predict(X_test)
Get_score(y_pred, Y_test)

The Spearman's correlation coefficient is: 0.277
The Spearman's correlation coefficient is: 0.101


Random Forest ensemble model

In [116]:
rf = RandomForestRegressor(n_estimators=100,random_state=42,verbose=2)
rf.fit(X_train,Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.9min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=2, warm_start=False)

In [117]:
rf_pred = rf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [118]:
Get_score(rf_pred, Y_test)

The Spearman's correlation coefficient is: 0.348
The Spearman's correlation coefficient is: 0.095


Building Neural Models from Keras

In [120]:
# add dropout - Helps prevent overfitting
# add regularizers - Regularizers allow to apply penalties on layer parameters or layer activity during optimization.
# activations - Used to determine the output of network. It maps the resulting values in between 0 to 1 or -1 to 1 etc. (depending upon the function).
model = Sequential() # The Sequential model is a linear stack of layers.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005),input_shape=(645,))) # Just your regular densely-connected NN layer.
model.add(layers.Dropout(0.5)) #Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training; helps prevent overfitting.
model.add(layers.Dense(10,activation='relu',kernel_regularizer=regularizers.l2(0.0005)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='sigmoid'))

In [121]:
# compile the model 
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])
# training the model 
history = model.fit(X_train,Y_train,epochs=20)
predictions = model.predict(X_test)
Get_score(predictions, Y_test)

Epoch 1/20
150/150 [==============================] - 1s 1ms/step - loss: 0.1063 - accuracy: 0.5049
Epoch 2/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0563 - accuracy: 0.5314
Epoch 3/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0362 - accuracy: 0.5387
Epoch 4/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0252 - accuracy: 0.5752
Epoch 5/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0204 - accuracy: 0.6257
Epoch 6/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0177 - accuracy: 0.6801
Epoch 7/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0159 - accuracy: 0.7115
Epoch 8/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0154 - accuracy: 0.7067
Epoch 9/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0148 - accuracy: 0.6986
Epoch 10/20
150/150 [==============================] - 0s 1ms/step - loss: 0.0145 - accuracy: 0.7057

SVR model

In [122]:
st_X = StandardScaler()
st_Y = StandardScaler()
st_X_train = st_X.fit_transform(st_X_train)
st_Y_train = st_Y.fit_transform(st_Y_train)
lt_X = StandardScaler()
lt_Y = StandardScaler()
lt_X_train = lt_X.fit_transform(lt_X_train)
lt_Y_train = lt_Y.fit_transform(lt_Y_train)

In [123]:
st_regressor = SVR(kernel = 'rbf')
st_regressor.fit(st_X_train, st_Y_train)
lt_regressor = SVR(kernel = 'rbf')
lt_regressor.fit(lt_X_train,lt_Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [124]:
st_pred = st_regressor.predict(st_X_test)
st_pred = st_Y.inverse_transform(st_pred)
lt_pred = lt_regressor.predict(lt_X_test)
lt_pred = lt_Y.inverse_transform(lt_pred)

In [125]:
Get_score(st_pred, st_Y_test)
Get_score(lt_pred, lt_Y_test)

The Spearman's correlation coefficient is: 0.242
The Spearman's correlation coefficient is: 0.099


XGBoost model

In [149]:
params = {'n_estimators':650, 'max_depth':12, 'min_samples_split':2, 'learning_rate':0.01, 'loss':'lad'}
xgb_st = xgb.XGBRegressor(**params)
xgb_lt = xgb.XGBRegressor(**params)
xgb_st.fit(st_X_train, st_Y_train)
xgb_lt.fit(lt_X_train, lt_Y_train)

[14:40:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:48:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, loss='lad',
             max_delta_step=0, max_depth=12, min_child_weight=1,
             min_samples_split=2, missing=None, n_estimators=650, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [150]:
Get_score(xgb_st.predict(st_X_test), st_Y_test)
Get_score(xgb_lt.predict(lt_X_test), lt_Y_test)

The Spearman's correlation coefficient is: 0.337
The Spearman's correlation coefficient is: 0.087


##Using the best performed model over the test set

Loading the ground truth file from the Dev set

In [157]:
# load the ground truth template
GT_t = pd.read_csv('./Test-set/Ground-truth_test/ground_truth_template.csv')
vid = GT_t.video.values  # list the video filenames from the ground truth template

In [159]:
GT_t.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,7494,NaN,33,NaN,12
1,7495,NaN,34,NaN,10
2,7496,NaN,32,NaN,13
3,7497,NaN,33,NaN,10
4,7498,NaN,33,NaN,10


Loading the features from test set into a python dataframe from the drive (one time load)

In [173]:
# Extracting the C3D test feature
c3d_features_t = pd.DataFrame(columns=['video','C3D'])
for i in range(2000):
  c3d_features_t = c3d_features_t.append({'video': vid[i], 'C3D': read_C3D('./Test-set/C3D_test/video' + str(vid[i]) + '.txt')}, ignore_index=True)
c3d_features_t.shape

(2000, 2)

In [178]:
# Extracting the HMP test feature
hmp_features_t = pd.DataFrame(columns=['video','HMP'])
for i in range(2000):
  hmp_features_t = hmp_features_t.append({'video': vid[i], 'HMP': read_HMP('./Test-set/HMP_test/video' + str(vid[i]) + '.txt')}, ignore_index=True)
hmp_features_t.shape

(2000, 2)

In [202]:
# Extracting the LBP test feature
lbp_features1_t = pd.DataFrame(columns=['video','LBP'])
lbp_features2_t = pd.DataFrame(columns=['video','LBP'])
lbp_features3_t = pd.DataFrame(columns=['video','LBP'])
for i in range(2000):
  lbp_features1_t = lbp_features1_t.append({'video': vid[i], 'LBP': read_LBP('./Test-set/LBP_test/video' + str(vid[i]) + '-0.txt')}, ignore_index=True)
  lbp_features2_t = lbp_features2_t.append({'video': vid[i], 'LBP': read_LBP('./Test-set/LBP_test/video' + str(vid[i]) + '-56.txt')}, ignore_index=True)
  lbp_features3_t = lbp_features3_t.append({'video': vid[i], 'LBP': read_LBP('./Test-set/LBP_test/video' + str(vid[i]) + '-112.txt')}, ignore_index=True)
lbp_features1_t.shape
lbp_features2_t.shape
lbp_features3_t.shape

(2000, 2)

Saving and Loading the extracted features using the pickle files

In [ ]:
#----------- Saving the test features to the drive ------------

os.chdir('/content/drive/MyDrive')
# Saving the C3D feature in Google Drive to reuse later  
c3d_features_t.to_pickle('./C3D_Features_t.pickle')
# Saving the HMP feature in Google Drive to reuse later  
hmp_features_t.to_pickle('./HMP_Features_t.pickle')
# Saving the LBP 0 frame feature in Google Drive to reuse later  
lbp_features1_t.to_pickle('./LBP_0Frame_Features_t.pickle')
# Saving the LBP 56 frame feature in Google Drive to reuse later  
lbp_features2_t.to_pickle('./LBP_56Frame_Features_t.pickle')
# Saving the LBP 112 frame feature in Google Drive to reuse later  
lbp_features3_t.to_pickle('./LBP_112Frame_Features_t.pickle')
# Changing the working directory
os.chdir('/content/drive/MyDrive/CA684_Assignment/')

In [ ]:
#----------- Loading the test features from the drive ------------

os.chdir('/content/drive/MyDrive')
# Load C3D feature back in memory session
c3d_features_t = pd.read_pickle('./C3D_Features_t.pickle')
# Load HMP feature back in memory session
hmp_features_t = pd.read_pickle('./HMP_Features_t.pickle')
# Load LBP frame features back in memory session
lbp_0frame_features_t = pd.read_pickle('./LBP_0Frame_Features_t.pickle')
lbp_56frame_features_t = pd.read_pickle('./LBP_56Frame_Features_t.pickle')
lbp_112frame_features_t = pd.read_pickle('./LBP_112Frame_Features_t.pickle')
# Changing the working directory
os.chdir('/content/drive/MyDrive/CA684_Assignment/')

Extracting the values of features from 1-D numpy array

In [193]:
# C3D feature value extracted
sequences = c3d_features_t['C3D']
C3D_seq_t = np.zeros((len(sequences),len(c3d_features_t.loc[0,'C3D'])))
for i in range(len(sequences)):
    n = len(sequences[i])
    C3D_seq_t[i,-n:] = sequences[i]
C3D_seq_t.shape

(2000, 101)

In [194]:
# HMP feature value extracted
sequences = hmp_features_t['HMP']
HMP_seq_t = np.zeros((len(sequences),len(hmp_features_t.loc[0,'HMP'])))
for i in range(len(sequences)):
    n = len(sequences[i])
    HMP_seq_t[i,-n:] = sequences[i]
HMP_seq_t.shape

(2000, 6075)

In [209]:
# LBP feature value extracted
LBP_seq_t = np.concatenate((LBP_Frame(lbp_0frame_features_t),LBP_Frame(lbp_56frame_features_t),LBP_Frame(lbp_112frame_features_t)),axis = 1)
LBP_seq_t.shape

(2000, 366)

PCA technique to reduce the dimensionality of HMP feature

In [222]:
from sklearn.decomposition import PCA
# preserving the 99.9% of the variance i.e. reduced components explains 99.9% variance of the data 
pca = PCA(n_components=178)
HMP_seq_PCA_t = pd.DataFrame(pca.fit_transform(HMP_seq_t))
HMP_seq_PCA_t.shape

(2000, 178)

Merging the feature set 

In [223]:
# merging the C3D and reduced HMP feature set
C3D_HMP_t = np.concatenate((C3D_seq_t,HMP_seq_PCA_t), axis = 1)

# merging the C3D and LBP feature set
C3D_LBP_t = np.concatenate((C3D_seq_t,LBP_seq_t), axis = 1)

# merging the C3D, LBP and reduced HMP feature set
C3D_LBP_HMP_t = np.concatenate((C3D_seq_t,LBP_seq_t,HMP_seq_PCA_t), axis = 1)

print(C3D_HMP_t.shape)
print(C3D_LBP_t.shape)
print(C3D_LBP_HMP_t.shape)

(2000, 279)
(2000, 467)
(2000, 645)


Applying the best performing models to 6000 video training data

In [224]:
# Setting up the DV (Y) and IDVs (X) along with train test split

st_Y = GT[['short-term_memorability']].values
lt_Y = GT[['long-term_memorability']].values

# short-term values
st_X_train = C3D_LBP_HMP
st_X_test = C3D_LBP_HMP_t
st_Y_train = st_Y

# long-term values
lt_X_train = C3D_HMP
lt_X_test = C3D_HMP_t
lt_Y_train = lt_Y


In [219]:
# Random Forest is the best model performed in Short-term video memorability using C3D, HMP & LBP video features
rf = RandomForestRegressor(n_estimators=100,random_state=42,verbose=2)
rf.fit(st_X_train,st_Y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.8min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=2, warm_start=False)

In [225]:
st_pred = rf.predict(st_X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [230]:
# updating the short-term predictions in ground truth template
GT_t['short-term_memorability'] = st_pred

In [233]:
# Support Vector Regressor is the best model performed in Long-term video memorability using C3D & HMP video features
svg_X = StandardScaler()
svg_Y = StandardScaler()
lt_X_train = svg_X.fit_transform(lt_X_train)
lt_Y_train = svg_Y.fit_transform(lt_Y_train)

In [234]:
lt_regressor = SVR(kernel = 'rbf')
lt_regressor.fit(lt_X_train,lt_Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [236]:
lt_pred = lt_regressor.predict(lt_X_test)
lt_pred = svg_Y.inverse_transform(lt_pred)

In [240]:
# updating the long-term predictions in ground truth template
GT_t['long-term_memorability'] = lt_pred

In [242]:
# Saving the Ground Truth csv on the drive
os.chdir('/content/drive/MyDrive')
# Saving the C3D feature in Google Drive to reuse later  
GT_t.to_csv('./Jai_Warde_20210699_Predictions.csv')